In [1]:
import xml.etree.ElementTree as ET
import os
from glob import glob
import cv2
import matplotlib.pyplot as plt

In [2]:
XML_PATH = '../../dataset/tymczasowy2/'
IMAGE_PATH = '../../dataset/tymczasowy/'

CLASSES_PATH = '../../class_names/butelka_classes.txt'
DEST_PATH = '../../dataset/tymczasowy3'

'''loads the classes'''
def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names


classes = get_classes(CLASSES_PATH)
assert len(classes) > 0, 'no class names detected!'
print(f'num classes: {len(classes)}')


num classes: 2


In [3]:
def convert_xmls(directory, num_files=None):
    num_files_converted = 0
    
    try:
        os.mkdir(os.path.join(DEST_PATH, directory))
    except OSError as error:
        print(error)    
    
    processed_images = glob(os.path.join(DEST_PATH, directory, '*'))
        
    for path in glob(os.path.join(XML_PATH, directory, '*.xml')):
        if(num_files != None and num_files_converted >= num_files):
            break
        in_file = open(path)

        # Parse .xml file
        tree = ET.parse(in_file)
        root = tree.getroot()
        # Write object information to .txt file
        
        file_name = root.find('filename').text
        print(file_name)
        image_name = path.split('/')[-1].split('.')[0]
        dest_path = os.path.join(DEST_PATH, directory, image_name + ".JPG") 
        
        if(dest_path in processed_images):
            continue
        try:
            obj = next(root.iter('object'))
            cls = obj.find('name').text 
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
            img = cv2.imread(os.path.join(IMAGE_PATH, directory, file_name), cv2.IMREAD_UNCHANGED)
            if(img.shape[0] < img.shape[1]):
                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            
            b = [int(bound * 3024/312) for bound in b]
            img = img[b[1]:b[3], b[0]:b[2]]
            
            cv2.imwrite(os.path.join(DEST_PATH, directory, file_name), img)
        
            num_files_converted = num_files_converted + 1
        except Exception as error:
            print(error)

In [10]:
dirs = ['prawidlowa']

In [12]:
for dir in dirs:
    convert_xmls(dir)

[Errno 17] File exists: '../../dataset/tymczasowy3/prawidlowa'
IMG_3948.JPG


In [6]:
b = (5, 123, 5, 11)

In [7]:
b

(5, 123, 5, 11)

In [5]:
b = [bound * 3024/312 for bound in b]

In [6]:
b

[48.46153846153846, 1192.1538461538462, 48.46153846153846, 106.61538461538461]